In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0002780.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0006524.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/000304.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0007130.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0004831.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0003379.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0001978.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0004880.jpg
/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images/normal_clothes/0007124.jpg
/kaggle/input/normal

In [3]:
import os
ORIG_INPUT_DATASET = "/kaggle/input/normal-vs-camouflage-clothes/8k_normal_vs_camouflage_clothes_images"
BASE_PATH = "camo_not_camo"

TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

In [4]:
TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.1
CLASSES = ["camouflage_clothes", "normal_clothes"]

In [5]:
INIT_LR = 1e-4
BS = 32
NUM_EPOCHS = 20
MODEL_PATH = "military_camo_detector.model"

In [6]:
!pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25861 sha256=29bd925ac443f673f112c19bef7c5f153e9ecb48bf0fb2e7df3b72f3a3639885
  Stored in directory: /root/.cache/pip/wheels/35/e4/69/cb99d996d14a2971b79b990d68b05a17d58ce530ff96090dfc
Successfully built imutils


In [7]:
from imutils import paths
import random
import shutil

In [8]:
imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)
# computing training and testing split
i = int(len(imagePaths) * TRAIN_SPLIT)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]
# part of the training data for validation
i = int(len(trainPaths) * VAL_SPLIT)
valPaths = trainPaths[:i]
trainPaths = trainPaths[i:]
# defining the datasets
datasets = [
	("training", trainPaths, TRAIN_PATH),
	("validation", valPaths, VAL_PATH),
	("testing", testPaths, TEST_PATH)
]

In [9]:
os.getcwd()

'/kaggle/working'

In [10]:
for (dType, imagePaths, baseOutput) in datasets:
	print("[INFO] building '{}' split".format(dType))
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)
	for inputPath in imagePaths:
		filename = inputPath.split(os.path.sep)[-1]
		label = inputPath.split(os.path.sep)[-2]
		labelPath = os.path.sep.join([baseOutput, label])
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] 'creating camo_not_camo/training' directory
[INFO] 'creating camo_not_camo/training/normal_clothes' directory
[INFO] 'creating camo_not_camo/training/camouflage_clothes' directory
[INFO] building 'validation' split
[INFO] 'creating camo_not_camo/validation' directory
[INFO] 'creating camo_not_camo/validation/camouflage_clothes' directory
[INFO] 'creating camo_not_camo/validation/normal_clothes' directory
[INFO] building 'testing' split
[INFO] 'creating camo_not_camo/testing' directory
[INFO] 'creating camo_not_camo/testing/normal_clothes' directory
[INFO] 'creating camo_not_camo/testing/camouflage_clothes' directory


In [11]:
import matplotlib
matplotlib.use("Agg")
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [12]:
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))
print("train ",totalTrain, " val ",totalVal," test ",totalTest)

train  10731  val  1192  test  3975


In [13]:
trainAug = ImageDataGenerator(
	rotation_range=25,
	zoom_range=0.1,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.2,
	horizontal_flip=True,
	fill_mode="nearest")
valAug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [14]:
trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
# validation generator
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
# testing generator
testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 10731 images belonging to 2 classes.
Found 1192 images belonging to 2 classes.
Found 3975 images belonging to 2 classes.


In [15]:
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)
for layer in baseModel.layers:
	layer.trainable = False

[INFO] preparing model...
94765736/94765736 [==============================] - 1s 0us/step


In [16]:
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
print("[INFO] training model...")
H = model.fit(
	trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	epochs=NUM_EPOCHS)

[INFO] training model...
Epoch 1/20
335/335 [==============================] - 318s 912ms/step - loss: 0.2066 - accuracy: 0.9213 - val_loss: 0.0809 - val_accuracy: 0.9713
Epoch 2/20
335/335 [==============================] - 320s 956ms/step - loss: 0.0844 - accuracy: 0.9721 - val_loss: 0.0700 - val_accuracy: 0.9772
Epoch 3/20
335/335 [==============================] - 300s 896ms/step - loss: 0.0716 - accuracy: 0.9777 - val_loss: 0.0692 - val_accuracy: 0.9738
Epoch 4/20
335/335 [==============================] - 297s 887ms/step - loss: 0.0575 - accuracy: 0.9807 - val_loss: 0.0577 - val_accuracy: 0.9780
Epoch 5/20
335/335 [==============================] - 295s 880ms/step - loss: 0.0570 - accuracy: 0.9816 - val_loss: 0.0691 - val_accuracy: 0.9730
Epoch 6/20
335/335 [==============================] - 301s 899ms/step - loss: 0.0490 - accuracy: 0.9828 - val_loss: 0.0492 - val_accuracy: 0.9831
Epoch 7/20
335/335 [==============================] - 298s 889ms/step - loss: 0.0445 - accuracy: 0.

In [19]:
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BS) + 1)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))

print("[INFO] saving model...")
model.save(MODEL_PATH, save_format="h5")

[INFO] evaluating network...


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  after removing the cwd from sys.path.


                    precision    recall  f1-score   support

camouflage_clothes       0.99      0.98      0.98      2007
    normal_clothes       0.98      0.99      0.98      1968

          accuracy                           0.98      3975
         macro avg       0.98      0.98      0.98      3975
      weighted avg       0.98      0.98      0.98      3975

[INFO] saving model...


In [21]:
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
#plt.savefig(args["plot"])